In [3]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import json
import psycopg2
from psycopg2.extras import execute_values

In [63]:
# Connect to ExCompany database on PostgreSQL

db_connection = psycopg2.connect(
    dbname = "ExCompany", 
    user = "postgres",
    password = "<password>",
    host = "localhost",
    port = 5432)

# Create a cursor object to execute SQL queries
cursor = db_connection.cursor()

In [5]:
# Create a Faker instance
fake = Faker()

In [9]:
# Define area code for company phone numbers
company_area_code = 312

# Define the bounds for latitude and longitude of Chicago (where employees live)
chicago_latitude_bounds = (41.6445, 42.023)
chicago_longitude_bounds = (-87.9401, -87.524)

# Generate random data for each column
num_rows = 100  # Number of rows
employee_data = []
for row in range(num_rows):
    employee_id = fake.unique.random_int(min = 100000, max = 999999) # Random 6-digit employee ID
    first_name = fake.first_name()
    last_name = fake.last_name()
    age = int(np.clip(np.random.normal(loc = 35, scale = 10), 18, 65)) # Random age between 18 and 65 drawn from normal distribution with mean = 35, sd = 10
    rating = max(0, min(5, round(np.random.normal(loc = 3, scale = 1), 2))) # Random rating between 0 and 5 drawn from normal distribution with mean = 3, sd = 1
        
    # Contact info for JSON and BJSON columns
    email = f"{first_name.lower()}.{last_name.lower()}@company.com" # Generate email based on name
    phone_number = f"{company_area_code}-555-{fake.unique.random_int(min = 1000, max = 9999)}" # Generate phone number with fixed area code and random extension number

    # Create JSON and BJSON contact info
    json_contact_info = {
        'phone': phone_number,
        'email': email,
    }
    bjson_contact_info = json.dumps(json_contact_info) # Serialize BJSON data to string using json.dumps()

    # Info for address geometry column (drawn from uniform distribution)
    latitude = random.uniform(chicago_latitude_bounds[0], chicago_latitude_bounds[1])
    longitude = random.uniform(chicago_longitude_bounds[0], chicago_longitude_bounds[1])
    address = f"POINT({longitude} {latitude})"  # Random point within Chicago bounds

    employee_data.append((employee_id, first_name, last_name, age, rating, json.dumps(json_contact_info), json.dumps(bjson_contact_info), address))
(employee_data[0][0], employee_data[0][3])

(794819, 38)

In [66]:
# Define the INSERT query
insert_query = """INSERT INTO employees (employee_id, first_name, last_name, age, rating, json_contact_info, bjson_contact_info, address)
                  VALUES %s"""

# Execute the INSERT query
execute_values(cursor, insert_query, employee_data)

# Commit the transaction
db_connection.commit()

In [ ]:
insert_query_full = """INSERT INTO employees (employee_id, first_name, last_name, age, rating, json_contact_info, bjson_contact_info, address)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""

for row in employee_data:
    cursor.execute(insert_query_full, row)

In [61]:
# Define the DELETE query
delete_query = "DELETE FROM employees"

# Execute the DELETE query
cursor.execute(delete_query)

# Commit the transaction and close the connection
db_connection.commit()
cursor.close()
db_connection.close()

In [8]:
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9
